In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
driver = webdriver.Chrome()

driver.get("https://www.binghamton-ny.gov/home")

from selenium.webdriver.common.action_chains import ActionChains
government_tab = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Government']")))

departments_link = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Departments']")))

ActionChains(driver).move_to_element(government_tab).click(departments_link).perform()

try:

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[text()='Personnel / Civil Service']"))
    )
    personnel_link = driver.find_element(By.XPATH, "//a[@href='https://www.binghamton-ny.gov/government/departments/personnel-civil-service']")
    personnel_link.click()
    employment_link = driver.find_element(By.XPATH, "//a[@href='https://www.binghamton-ny.gov/government/departments/personnel-civil-service/employment']")
    employment_link.click()

finally:

    pass


soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find('table', {'class': 'listtable responsive-table-data-mb'})
rows = table.findAll('tr')
data = []

for row in rows[1:]:

    cols = row.findAll('td')

    job_title = cols[0].text.strip()

    job_type = cols[1].text.strip()

    job_closing_date = cols[2].text.strip()

    job_url = cols[0].find('a')['href']

    data.append({'Job Title': job_title, 'Job Type': job_type, 'Closing Date': job_closing_date})

df = pd.DataFrame(data)

df.to_csv('job_openings.csv', index=False)

df_read = pd.read_csv('job_openings.csv')


soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.back()
driver.quit()